In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 1. Load the Data
data = pd.read_csv('realistic_rice_seedling_data.csv')

In [3]:
data.columns

Index(['Day', 'Temperature (°C)', 'Humidity (%)', 'Light Intensity (lux)',
       'CO2 Levels (ppm)', 'Water Supply (L/day)',
       'Nutrient Concentration (%)', 'Soil Moisture (%)',
       'Seedling Height (cm)', 'Leaf Count', 'Root Length (cm)',
       'Plant Weight (g)', 'Leaf Area (cm²)', 'Health Score'],
      dtype='object')

In [4]:
# Features: Environmental conditions
features = data[['Temperature (°C)', 'Humidity (%)', 'Light Intensity (lux)', 
                 'CO2 Levels (ppm)', 'Water Supply (L/day)', 
                 'Nutrient Concentration (%)', 'Soil Moisture (%)']]

In [5]:
targets = pd.DataFrame({
    'Optimal Temperature (°C)': data['Temperature (°C)'] + np.random.uniform(0, 2, len(data)),
    'Optimal Humidity (%)': data['Humidity (%)'] + np.random.uniform(-5, 5, len(data)),
    'Optimal Light Intensity (lux)': data['Light Intensity (lux)'] + np.random.uniform(-500, 500, len(data)),
    'Optimal CO2 Levels (ppm)': data['CO2 Levels (ppm)'] + np.random.uniform(-20, 20, len(data)),
    'Optimal Water Supply (L/day)': data['Water Supply (L/day)'] + np.random.uniform(-0.1, 0.1, len(data)),
    'Optimal Nutrient Concentration (%)': data['Nutrient Concentration (%)'] + np.random.uniform(-0.5, 0.5, len(data)),
    'Optimal Soil Moisture (%)': data['Soil Moisture (%)'] + np.random.uniform(-5, 5, len(data))
})

In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)
scaled_targets = scaler.fit_transform(targets)

In [7]:
# Reshape for LSTM [samples, time steps, features]
X = np.reshape(scaled_features, (scaled_features.shape[0], 1, scaled_features.shape[1]))
y = scaled_targets

In [8]:
# 3. Build the LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(64, activation='relu'))  # Hidden layer
model.add(Dense(y.shape[1], activation='linear'))  # Output layer: one node for each optimal condition

C:\Users\lenovo\OneDrive\Desktop\Major Project\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
from tensorflow.keras.optimizers import Adam
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')


In [10]:
# 4. Train the Model
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - loss: 0.3183 - val_loss: 0.1751
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2220 - val_loss: 0.1131
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1309 - val_loss: 0.0624
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0592 - val_loss: 0.0573
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0465 - val_loss: 0.0510
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0417 - val_loss: 0.0411
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0378 - val_loss: 0.0366
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0345 - val_loss: 0.0356
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0326 - val_loss: 0.0319
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0292 - val_loss: 0.0286
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0260 - val_loss: 0.0252
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step -

In [11]:
# 5. Make Predictions for Optimization
predicted_conditions = model.predict(X)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


In [24]:
# Rescale the predictions back to original scale
predicted_conditions = scaler.inverse_transform(predicted_conditions)


In [25]:
# Display predicted optimal conditions for first few rows
print("Predicted Optimal Conditions:")
print(predicted_conditions[:5])


Predicted Optimal Conditions:
[[2.44514355e+04 2.85943625e+05 2.37358128e+12 3.16014400e+07
  5.91671515e+00 8.89926834e+01 2.58930723e+04]
 [2.33071758e+04 2.71244375e+05 2.23375288e+12 3.49690480e+07
  6.23303223e+00 7.11050644e+01 1.89773848e+04]
 [2.50009648e+04 2.71088031e+05 2.31484948e+12 3.15253900e+07
  3.21536088e+00 7.28058777e+01 2.64512539e+04]
 [2.50060469e+04 2.69330344e+05 2.52947517e+12 3.95254880e+07
  6.22851038e+00 1.00628555e+02 2.95546152e+04]
 [2.56213164e+04 2.82602125e+05 2.33685726e+12 3.28845960e+07
  2.83352232e+00 4.61566238e+01 2.04597090e+04]]
